<a href="https://colab.research.google.com/github/MasahiroAraki/SpeechRecognition/blob/master/Python/chap09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 例題9.1

scikit-learn付属のirisデータについて、SVCで学習した結果を分割学習法と交差確認法で評価し、評価法の特性を把握せよ。

データの準備と表示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.svm import SVC

iris = load_iris()
X = iris.data  #特徴は4次元
y = iris.target #クラスは3つ
plt.scatter(X[:, 0], X[:, 1], marker='o', c=y)
plt.show()

分割学習法

データを分割するメソッド train_test_split は、実行毎に乱数で分割を変える。分割を変えて評価を何度か実行せよ。

In [ ]:
from sklearn.model_selection import train_test_split





交差確認法

交差確認法に用いる分割は、ShuffleSplit（実行毎に乱数で分割を変える）を用いて行う。分割数を変えて分割学習法自体を何度か実行して評価値のぶれを観察せよ。

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
